In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch
import copy

In [ ]:
from src.training.SmallRawDataset import SmallRawDataset
# from src.Restorer.Cond_NAF import make_model, make_full_model, make_full_model_RGGB

from src.training.ShadowAwareLoss import ShadowAwareLoss
from src.training.VGGFeatureExtractor import VGGFeatureExtractor
from src.training.sparse_loop import train_one_epoch, visualize
from src.training.rggb_loop import train_one_epoch_rggb, visualize


from src.training.utils import apply_gamma_torch


In [ ]:
device= 'mps'

batch_size = 2
lr = 1e-4 * batch_size / 4
# lr = 1e-3 * batch_size / 32
clipping =  1e-2

num_epochs = 75
val_split = 0.2

In [ ]:
dataset = SmallRawDataset('/Volumes/EasyStore/RAWNIND/JPEGs/Cropped_JPEG_high_quality/', 'align.csv', crop_size=256)


# Split dataset into train and val
val_size = int(len(dataset) * val_split)
train_size = len(dataset) - val_size
torch.manual_seed(42)  # For reproducibility
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Set the validation dataset to use the same crops
val_dataset = copy.deepcopy(val_dataset)
val_dataset.dataset.validation = True

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# model_name = '/Volumes/EasyStore/models/Cond_NAF_original_null.pt'
# model = make_full_model_RGGB(model_name=None)
# model = model.to(device)



In [ ]:
# # model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_300_full.pt'
# # model = make_full_model(model_name=model_name)
# # model = model.to(device)

# model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_300_full_RGGB.pt'
# model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_0_full_RGGB.pt'

# model = make_full_model_RGGB(model_name=model_name)
# model = model.to(device)


In [ ]:
from src.Restorer.Cond_NAF import make_model, make_full_model, make_full_model_RGGB

model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_300_full_RGGB.pt'
model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_0_full_RGGB.pt'

model = make_full_model_RGGB(model_name=model_name)
model = model.to(device)


from src.Restorer.Cond_CHASPA import  make_full_model_RGGB


model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_0_full_RGGB_CHASPA.pt'

model = make_full_model_RGGB(model_name=model_name)
model = model.to(device)


In [ ]:
# model_name = '/Volumes/EasyStore/models/Restorer_train_vgg_relu_300_full.pt'
# model = make_full_model(model_name=model_name)
# # model = model.to(device)

In [ ]:
model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

vfe = VGGFeatureExtractor(config=((1, 64), (1, 128), (1, 256), (1, 512), (1, 512),), 
                          feature_layers=[14], 
                          activation=nn.ReLU
                          )
vfe = vfe.to(device)

loss_fn = ShadowAwareLoss(
    alpha=0.2,
    beta=5.0,
    l1_weight=0.16,
    ssim_weight=0.84,
    tv_weight=0.0,
    vgg_loss_weight=0,
    apply_gamma_fn=apply_gamma_torch,
    vgg_feature_extractor=vfe,
    device=device,
)

In [ ]:
for epoch in range(num_epochs):
    train_one_epoch(epoch, model, optimizer, model_name, train_loader, device, loss_fn, clipping, log_interval = 10, sleep=0.0, rggb=True)